In [1]:
!pip install numpy==1.24.4

In [2]:
!pip install emoji

In [3]:
!pip install pandas

In [4]:
!pip install seaborn matplotlib

In [5]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import emoji
%matplotlib inline

In [6]:
#importing rdkit
!pip install rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit import RDConfig
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit import DataStructs
from rdkit.Chem.Subshape import SubshapeBuilder, SubshapeAligner, SubshapeObjects

In [7]:
!pip install scikit-learn

In [8]:
#importing sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR


In [9]:
# Import mordred and padelpy
!pip install mordred
!pip install padelpy
import padelpy

#from mordred import Calculator, descriptors
from mordred import Calculator, descriptors


In [10]:
# Load dataset
df = pd.read_csv('reduced.csv')


In [11]:
from rdkit.Chem import MACCSkeys

In [12]:
print(df.head)

<bound method NDFrame.head of                       SMILES_Solute  Temperature_K  Solvent SMILES_Solvent  \
0      CCCCCCCCCCCCCCCCCCCC(=O)OCCO         311.25  ethanol            CCO   
1      CCCCCCCCCCCCCCCCCCCC(=O)OCCO         314.65  ethanol            CCO   
2      CCCCCCCCCCCCCCCCCCCC(=O)OCCO         319.15  ethanol            CCO   
3      CCCCCCCCCCCCCCCCCCCC(=O)OCCO         322.15  ethanol            CCO   
4      CCCCCCCCCCCCCCCCCCCC(=O)OCCO         324.15  ethanol            CCO   
...                             ...            ...      ...            ...   
49995             CCOc1cc(C=O)ccc1O         308.15  acetone        CC(C)=O   
49996             CCOc1cc(C=O)ccc1O         313.15  acetone        CC(C)=O   
49997    Cn1c(=O)c2[nH]cnc2n(C)c1=O         288.02  acetone        CC(C)=O   
49998    Cn1c(=O)c2[nH]cnc2n(C)c1=O         298.15  acetone        CC(C)=O   
49999    Cn1c(=O)c2[nH]cnc2n(C)c1=O         308.15  acetone        CC(C)=O   

       Solubility(mole_fraction) 

In [13]:
# Extract solute and solvent SMILES
solute_smiles = df.iloc[:, 0]     # Solute SMILES from column 0
solvent_smiles = df.iloc[:, 3]    # Solvent SMILES from column 3

In [24]:
%%time
# Generate solute MACCS fingerprints
solute_molfps = []
for smi in solute_smiles:
    mol = Chem.MolFromSmiles(smi)
    if mol:
        fp = list(MACCSkeys.GenMACCSKeys(mol).ToBitString())
        fp = list(map(int, fp[1:]))    # Remove bit 0, map to int
    else:
        fp = [0] * 165                 # If invalid SMILES, use zero vector
    solute_molfps.append(fp)
solute_fps_df = pd.DataFrame(solute_molfps)
#solute_fps_df.columns = ['Solute_MACCS' + str(i) for i in range(1, 166)]


CPU times: user 45.2 s, sys: 489 ms, total: 45.7 s
Wall time: 46 s


In [25]:
solute_fps_df.head()

,0,1,2,3,4,5,6,7,8,9,...,156,157,158,159,160,161,162,163,164,165
0,0,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,0,0,1,0,0


In [26]:
%%time
# Generate solvent MACCS fingerprints
solvent_molfps = []
for smi in solvent_smiles:
    mol = Chem.MolFromSmiles(smi)
    if mol:
        fp = list(MACCSkeys.GenMACCSKeys(mol).ToBitString())
        fp = list(map(int, fp[1:]))    # Remove bit 0, map to int
    else:
        fp = [0] * 165                 # If invalid SMILES, use zero vector
    solvent_molfps.append(fp)
solvent_fps_df = pd.DataFrame(solvent_molfps)
#solvent_fps_df.columns = ['Solvent_MACCS' + str(i) for i in range(1, 166)]


CPU times: user 13 s, sys: 241 ms, total: 13.3 s
Wall time: 13.3 s


In [27]:
temperature = df.iloc[:, 1] 

In [29]:
X = pd.concat([solute_fps_df, temperature, solvent_fps_df], axis=1)

In [33]:
y = df.iloc[:, 5].to_numpy()


In [34]:
# Final outputs:
print("Feature matrix X shape:", X.shape)
print("Target vector y shape:", y.shape)

Feature matrix X shape: (50000, 333)
Target vector y shape: (50000,)


In [36]:
# Split the data randomly using train_test_split()
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=991)

In [37]:
from sklearn.linear_model import LinearRegression


# Train linear regression model
reg = LinearRegression()
reg.fit(X_train, Y_train)


TypeError: Feature names are only supported if all input features have string names, but your input has ['int', 'str'] as feature name / column name types. If you want feature names to be stored and validated, you must convert them all to strings, by using X.columns = X.columns.astype(str) for example. Otherwise you can remove feature / column names from your input data, or convert them all to a non-string data type.

In [38]:
X_train.columns = X_train.columns.astype(str)


In [39]:
reg = LinearRegression()
reg.fit(X_train, Y_train)


LinearRegression()

In [42]:
X_test.columns = X_test.columns.astype(str)


In [43]:
# Predictions using trained linnear regression model(reg)
Y_pred_train = reg.predict(X_train)  #predict for training data
Y_pred_test = reg.predict(X_test)    #predict for testing data

In [44]:
# Metrics
print('Linear Regression - Train R2: %.2f' % r2_score(Y_train, Y_pred_train))
print('Linear Regression - Test R2: %.2f' % r2_score(Y_test, Y_pred_test))


Linear Regression - Train R2: 0.12
Linear Regression - Test R2: 0.10


In [45]:
# MAE
print('MAE Train:', mean_absolute_error(Y_train, Y_pred_train))
print('MAE Test:', mean_absolute_error(Y_test, Y_pred_test))

MAE Train: 1.7145246381129895
MAE Test: 1.7778586489909354


In [46]:
%%time
# Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, Y_train)
Y_pred_train = model.predict(X_train)
Y_pred_test = model.predict(X_test)

CPU times: user 1min 37s, sys: 571 ms, total: 1min 37s
Wall time: 1min 38s


In [47]:
print('Random Forest - Train R2: %.2f' % r2_score(Y_train, Y_pred_train))
print('Random Forest - Test R2: %.2f' % r2_score(Y_test, Y_pred_test))

Random Forest - Train R2: 0.97
Random Forest - Test R2: 0.80


In [48]:
# MAE
print('MAE Train:', mean_absolute_error(Y_train, Y_pred_train))
print('MAE Test:', mean_absolute_error(Y_test, Y_pred_test))


MAE Train: 0.15658091272813823
MAE Test: 0.42891229675535597


In [49]:
# Split the data randomly using train_test_split()
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.4, random_state=991)

In [51]:
X_test.columns = X_test.columns.astype(str)


In [52]:
X_train.columns = X_train.columns.astype(str)


In [53]:
# Train linear regression model
reg = LinearRegression()
reg.fit(X_train, Y_train)


LinearRegression()

In [54]:
# Predictions using trained linnear regression model(reg)
Y_pred_train = reg.predict(X_train)  #predict for training data
Y_pred_test = reg.predict(X_test)    #predict for testing data

In [55]:
# Metrics
print('Linear Regression - Train R2: %.2f' % r2_score(Y_train, Y_pred_train))
print('Linear Regression - Test R2: %.2f' % r2_score(Y_test, Y_pred_test))


Linear Regression - Train R2: 0.11
Linear Regression - Test R2: 0.11


In [56]:
# MAE
print('MAE Train:', mean_absolute_error(Y_train, Y_pred_train))
print('MAE Test:', mean_absolute_error(Y_test, Y_pred_test))

MAE Train: 1.7182526499357724
MAE Test: 1.7526364292659902


In [57]:
%%time
# Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, Y_train)
Y_pred_train = model.predict(X_train)
Y_pred_test = model.predict(X_test)

CPU times: user 1min 11s, sys: 147 ms, total: 1min 12s
Wall time: 1min 12s


In [58]:
print('Random Forest - Train R2: %.2f' % r2_score(Y_train, Y_pred_train))
print('Random Forest - Test R2: %.2f' % r2_score(Y_test, Y_pred_test))

Random Forest - Train R2: 0.96
Random Forest - Test R2: 0.75


In [59]:
# MAE
print('MAE Train:', mean_absolute_error(Y_train, Y_pred_train))
print('MAE Test:', mean_absolute_error(Y_test, Y_pred_test))


MAE Train: 0.18135913167642181
MAE Test: 0.47149043840075827
